# Asymetric Criptography

## 1. Introduction

In this second lab of the subject of system administration and security, we will focus on how does it work asymetric criptography. As you know, cryptography is a very useful tool when computer security is desired; it can also be understood as a means to guarantee the confidentiality, integrity and availability properties of a system's resources. 

In the previous lab, we started to use symmetric criptography, in which you only need a key to encrypt and decrypt, basically as classical cryptography. With this type of cryptography we can guarantee confidentiality because only whoever has the secret key will be able to see the message. 

The problem with symmetric cryptography is that if I wanted to share secrets with m people, for each person I would have to generate a new secret key and the personal administration of all m keys would be chaotic. Another problem associated with this type of cryptography is how I share with another person in a confidential and integrated way the secret key. These problems are solved to some extent with asymmetric cryptography.


In Asymetric criptography dissapears this symmetry by adding a public key. Basically, the public key is used for encrypt data, wheread the other one, the private key is used to decrypt that data. 

Adding this public key, we solve many problems, some metioned previously:
- **Safe distribution of keys**: In symmetric criptography you need a safe channel to share the private key. With asymmetric, using a public key to encrypt and a private key to decrypt, it is deleted the necessity to share private key.

- **Keys scalability**: In symmetric criptography, the number of keys grows exponentially with the number of users we need to share key. With asymmetric, each user only needs a pair of keys(public and private), simplifying the management.

- **Authenticity**: In symmetric criptography, anyone with the key can encrypt, making it impossible to verify who sent the message. With asymmetric, digital signatures with the private key guarantee the authenticity of the sender.

- **No long-term confidentiality**: In symmetric criptography, if a symmetric key is compromised, all messages encrypted with it are exposed. With asymmetric, the private key is not shared and messages encrypted with the public key remain secure

Apart from using asymetric Criptography, we will also use the Diffie-Hellman algorythm. The main objective of this algorythm is to achieve the exchange of a secret key via an unsafe channel like Internet. In order to understand Diffie-Hellman because it is quite hard mathematically, I will use the example with colours. 

### Exemple with colours

Imagine the situation that Sahoni and Ruben want to share a secret colour and for that, they follow the next algorythm: 
- Sahoni & Ruben start using the same colour that anybody can know. 
- Later, each one adds a secret colour that only they know(individually).
- After, they exchange the colours via a channel that anybody can know. 
- Finally, each one add the secret colour to the mixture that each one received. Then, both will have the same colour.

If you understand it good, you know now what it consist of Deffie Hellman algorythm. 

<img src="images/llustration-of-Diffie-Hellman-algorithm-with-colors.png" width="500" height="500" alt="Asymmetric Cryptography">

In our case, we will use this algorythm with number. Instead of using colours, we will use numbers in order to create a secret number or a password that it will be used to encrypt our messages. And why not to encrypt our messages with asymmetric criptography ? 

Basically because it is computationally **more expensive and slower** than symmetric criptography. For this reason, we will use both types.

# 2. Objective of the lab 

The objective of lab is to understand the fundamentals of asymmetric cryptography and Diffie Hellman by implementing a safe communication between two clients with both tools and sending a message to show its correct use. 


 In order to be done, the lab will be spread in four parts: 

- The first part, implement a function or algorythm to generate keys.
- The second part, implement a channel to exchange keys.
- The third part, implement a second channel to exchange the encrypted number and decrypt them.
- The fourth part, use the same channel to exchange a message with the secret number.

In this practice, you will be exploring the whole process of protecting information. In addition, the student is expected to analyse the importance of maintaining the secret key and experiment with the risks associated with using incorrect keys for decryption.

# 3. Development of the lab 

Before you start the lab, it is interesting you install a library. 

In [5]:
%pip install cryptography

  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------- ----------------- 1.8/3.2 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 10.8 MB/s eta 0:00:00
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


This library gives us tools to use to implement cryptography.

Once you have done it, you should import two libraries in order that the development goes properly.

In [3]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

- **rsa**: This import allows us to generate keys and also to encrypt and decrypt files.

- **serialization**: This import allow us to serialize and deserialize cryptographic keys.

In this lab, the decision of using rsa is due to: 
- It is easy to understand and use
- There's a high support for python libraries
- High compatibility and standardisation 
- Suitable for small data

I recommend to use this webpage to do the lab: https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/ 

### 3.1. Part I: Generation of keys

First of all, you will need to create the function to generate both keys. I suggest you that the the size of the key should be of 4096 bit in order you don't have any problem when you send a message. Moreover, the public_exponent parameter is 65537 because almost everyone use it.

In [4]:
def generate_keys_b():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096
        # Depending the size of the key, the encryption and decryption will be faster or slower
        # Also, the size of the key will determine the security of the key
        # Besides, the size of the key will determine the size of the encrypted message
    )
    public_key = private_key.public_key()
    return private_key, public_key

private_key_b, public_key_b = generate_keys_b()

Afterwards, you will need to convert that keys to another format. I suggest you that you find information about PEM format because with this format,  keys can be *readable*. Instead, if you want to pass it into another format, I suggest you using DER format where keys will in binary code.

In [5]:
public_pem_b = public_key_b.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

In [6]:
private_pem_b = private_key_b.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

In [ ]:
print("Public key:\n", public_pem_b.decode())
print("Private key:\n", private_pem_b.decode())

### 3.2. Part II: Implement a channel to exchange keys

On this second part, you will need to develop two vital points.
- Create a communication channel to exchange keys.
- Send the mix of numbers and save them locally.

Firstly, you will need to import the library socket in order you can use all its functions.

In [8]:
import socket
from cryptography.hazmat.backends import default_backend

The first import gives you tools to implement network communication. The second is a module from the cryptography library that is used for implementing specific cryptographics operations like key generation, signs, etc.

Later, you will to need to make a function to make the communication channel from client B to client A. In this case, client A (it won't the one that you wil develop here) will be the one who will wait for the connection from the other client. I suggest that you make the channel via TCP.

In [ ]:
def client_b_exchange():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(('localhost', 12345))
        s.sendall(public_pem_b)  # Send public key to A
        client_a_public = s.recv(1024)  # Receive public key from A
        print("Public A key received:\n", client_a_public.decode())
        
        
        client_a_public_key = serialization.load_pem_public_key(
            client_a_public,
            backend=default_backend()
        )
        

        return client_a_public_key

client_a_public = client_b_exchange()

If you did it correctly, you should receive the public key from A client.

### 3.3. Part III: Second channel development and sending of encrypted message key

In the last part of this lab, you will need to develop a second channel in order to send the encrypted number to B and after, A will send its mixed number. You need to know that you **CAN'T USE**  the same channel where you made the exchange of keys. The phases of this part are the following ones: 

- Sign the mixed number with A private key (Client A) and sign the mixed number with B private key (Client B)
- Use the public key to encrypt the mixed number (Client A & B)
- Create the channel to transmit the mixed number (Client A & B)
- Transmit the mixed numbers through the channel (Client A & B)
- Decrypt the mixed number that they were sent (Client A & B)

First of all, you will need to import two more libraries in order you can do properly this part. 

In [10]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

- **padding**: It allows us to protect data when you encrypt or decrypt it.
- **hashes**: It gives support for hashing algorythms. Basically, it maintains the integrity of our data.

After adding both libraries, first, you will need to do the function that makes possible to do the mix between the public numbers and the private number of client B.

In [12]:
def power(a,b,p):
    if b == 1:
        return a
    else:
        return pow(a, b) % p

In [13]:
public_num_a = 7
public_num_b = 13
private_num_b = 5

key_b_generated = power(public_num_a, private_num_b, public_num_b)
print("Key generated by B:", key_b_generated)

Key generated by B: 11


Once finished the mixed number generation function, you will need to code the part that allows you to sign the message with B private key.

In [14]:
# Sign the message
signature = private_key_b.sign(
    str(key_b_generated).encode(),
    padding.PSS(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

print("Firm generated:", signature)

Firm generated: b'\x01\xdf\xe4B\x9fC?S\xc2\xe7\tpT\xdc\xaa\x04;/\xf7\xf6\xb7\xb2r\xac,~\x7f\xac\xf1\x8bF\xd0\xe6@!\x11%\xd0B\xde\x99\x1e0\xe3\x1c\xa5\x95\xc8\xd9\x04\'\xeb\xbc\xfd\x13\x1b..OU\xe8U\xfb\x8f\xce\x85\x00\x8e\xe7\xd7\xe1u\t\x89\xfc\x9c\x18\x90\xe2\xce\xeeUl\xbc\xc5\x98\x94\xf0$\xce\x12)t\xdd\xee\x18B\x9f\xe4\xea\x94)U\xc1\x04\xe6\x19)\xde\xe7\x97\xbd\x1d\\\x7fe4\x02F\xdf\xd1\xd1\xb1\xd4,\x8bfM\xcd,\xb3\x9b\xe7~\x86fP#\xd1\\\xeb\xb4\x17\xb2\x1c\xe7\xe9\x07L\x1aN\x1a\x1f:wX\xac\x171\xad\xe4\x90c\\\x10\xf4\xbdV\x88\xbd\xa1\xd2\x13Q@x\xa5\xe1lBF[\xcb\xd0a\x0b}\xe4\x89\xce\x8d\x9e30\x8c,\xb8"\xa8\xd6\xf7\xa6\x97v\xe1\x94\xa8+\xf5U\xb2O\xffi\xbe\xe1\xc6\x07\x8d4\xe2>X-\x08\x14\xa6\x96}\xcc:\xdf\xc0\xed\x05hD"m\xa0\xd18Hh\x8d\xca\xb9S\xc7\xbf\x0c\xd6\xc3\xd4\x92\x12o }Xs!\xcd:\x8bt;\x01\xc4\xbdA*\xb9w\xa1\xa1;\x0c\x1cY\xecaq#\xef\x12\x87~\xe4\x13\x8eux\xcf\x12<&\xb5\x01\xcf^\x99\x9c~]\x96\x86\x19\xcf\xc9\x7fC\xd4!\xd5)\xb2\'\x8a{,H\xd1\x9f7\xd0\xbd/\x0b\t\x7f\xf7\n!P:\xf09\x92\xbb

From the signing process, there are several things that need to be explained: 
- PS5 Padding: Padding adds  randomness to prevent pattern-related attacks on signed data. It can be used or PS5 or PKCSv15. Normally, the recommended choice is PS5 for any new protocols or appplications. Then, PKCSv15 is usually used to support legacy protocols. 

- mgf : A function mask generator based on hash SHA256 algorythm. MGF1 generates random values that they are combined with the message to ensure a unique padding.

- salt-length: It stablishes the "jump" length (an additional random value used to ensure the randomness) 

- hashes.SHA256 : It is the hash algorythm to generate a summary of the message. The summary is compact and fixed representation of data. You can use different hashes from different families. In my case, I used this one because is enough for most of modern applications.

Now, you need to encrypt the message.

In [15]:
chunk_size = 214
data_to_encrypt = str(key_b_generated).encode() + b"||SIGNATURE||" + signature
encrypted_blocks = []

# Divide the message in chunks
for i in range(0, len(data_to_encrypt), chunk_size):
    chunk = data_to_encrypt[i:i + chunk_size]
        
    # Cypher the chunk
    encrypted_chunk = client_a_public.encrypt(
        chunk,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    print(len(encrypted_chunk))
    encrypted_blocks.append(encrypted_chunk)

print("Encrypted blocks:", encrypted_blocks)

512
512
512
Encrypted blocks: [b"\x0cG\xdc\x94\x9f\x0c\\k2v\xd6\xe7\xcd7&\xf3A\xa8m\x05Y\xd4\xa6\x862*\x9di\xc3^\xf1\xe1\xf6\x89u\x95\x1a\xedKR\x83#\x1c:\xd8\xcd$-\xe9\xa3}\xeb\xc5\xb58<\xb2\xa4\x99Z\r`7_\xc0\xd2\xb8\xe9\xbf\xcbk\x7f\xa7m\xf8\xa4\xc6\x19Dl\xa5\x86\xee\x86v\xfd\x83eVb@\x96\xd3F\x16\xce\x9d\xe7\xcf\xc9F\xd3Fb\x1c\x98\xb5y\xe7c\xeb2B\x8e\x0cA\xa1\xec\xe8\xb3\xcbj\x81L=\x81Cx\xb8\xfb\xd7\xe1D$1\x96W\xc1]r\x0fxV8\x92F\xb9n\xbd\rA\xc4\xfa\x14\xf3\xfd\xfe\xbe1\xc9_k\x1e\xdd\x05\xf5\x9e\x82\x05\xf8\xe5\xe6\xa9\xc3\xef\\\xebi\x0byO\xcf\xd4\xa3\xf2\xbe0\xc5\xff\x13\x02\xcd\xeb@\xde\xc5\xb6\xbc\xafZh7r\xa3\x9fS\xdb\x17)e\x89\x16'\x06\xe6\x1dA\xb3\x1b\xc5>/\xc9\xf3\x121*&P\xa4$O\r}\x83.$\x82\xa0H\xd0$\xf1\x86]W:\x95\x0197\xa15z\xe84vqx\xbc\x93\x1e\xc4\x11\xef\xb2p<\xe9\xef\xc8\xbc\x94\xc5 Q\x14?LA\xf7N\x00\xb1\xd8qE\xe9\x80\x02\xdc\x82,\xbfE\xf4\xf7\x944\xa8\xea\xbfL\x9bo\xbfp&\\C\x03\x8f\x8d\xc6y\xeeO\xf5\x10\x17\xabGl>\xca\\c\xf1\xad\xd46eIyW\xbbm\xb8wZ\x83\x0b\x9f}\x85\xe9\xab\

In order to encrypt the message, firstly, you need to create a string where you will need to concatenate the message with signature. Between the message and the signature, I recommend that you put a flag to separate the message and the signature. Afterwards, you may need to separate the whole message in chunks or blocks in order to not exceed the hash maximum permited length. 

From the encryption function, you should know that the message is encrypted the public key, meaning anyone can encrypt data. Then, when the message is sent and received by somebody, the receiver needs to decrypt the data with private key. Moreover, like signtures, RSA supports encryption with several different padding options. In our case, valid paddings for encryption can be OAEP or PKCS1v15. OAEP is the most recommended for any new protocols whereas PKCS1v15, it should be only used to support legacy protocols.

Later, you need to create the function that allows to receive and decrypt the message for client B.

In [17]:
def receive_and_decrypt_message(private_key_b):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('localhost', 12346))
        s.listen()
        print("Waiting for encrypted message...")
        conn, addr = s.accept()
        with conn:
            # Receive the encrypted message in parts
            encrypted_message_parts = []
            while True:
                part = conn.recv(1024)
                if not part:  # If there is no more data to receive, break the loop
                    break
                encrypted_message_parts.append(part)

            # Decrypt the message
            decrypted_data = b""
            for encrypted_part in encrypted_message_parts:
                decrypted_part = private_key_b.decrypt(
                    encrypted_part,
                    padding.OAEP(
                        mgf=padding.MGF1(algorithm=hashes.SHA256()),
                        algorithm=hashes.SHA256(),
                        label=None
                    )
                )
                decrypted_data += decrypted_part  # Gather the decrypted parts

            print("Decrypted message received")
            return decrypted_data

decrypted_data = receive_and_decrypt_message(private_key_b)

Waiting for encrypted message...
Decrypted message received


Basically, you need to open the channel and wait there is the connection request. After accepting the request, you start receiving the parts of the encrypted message and joining them. 

Once it has received every part, you will to create the code to decrypt the message in parts with the same features as it was encrypted. After every part has been decrypted, you will need to join everything another time.

Now, you need to change the roles from both clients. Client B will send the mixed number to client A.

In [18]:
# Send the encrypted message to Client A
def send_encrypted_message(encrypted_blocks):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(('localhost', 12346))  # Channel 2
        for block in encrypted_blocks:
            print(len(block))
            s.sendall(block)
        
        print("Message sent to Client B")

send_encrypted_message(encrypted_blocks)

512
512
512
Message sent to Client B


After both clients have received their mixed numbers, you will need to verify if the message was sent by the correct sender. Basically, you will need to separate the message and the signature. Later, you will need to use the function to verify both things.

In [19]:
# Separating the key from the signature
key_from_a, signature = decrypted_data.split(b"||SIGNATURE||")  
# Ensure that the key is the same
# Verify the signature

try:
    client_a_public.verify(
        signature,  
        key_from_a,    
        padding.PSS(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH 
        ),
        hashes.SHA256()
    )
    print("The firm is valid:", key_from_a.decode())  
except Exception as e:
    print("The firm is not valid:", e)  


The firm is valid: 5


### 3.4. Part IV: Transmission of encrypted messages with the Deffie-Hellman key

If the signature was right, no exception message will appear. Then, after transforming the number from string to integer, you need to create the secret number that will be the same for both clients. 

With this secret number, you will 

In [20]:
secret_key_num = power(int(key_from_a), private_num_b, public_num_b)
print("Secret number key generated by B:", secret_key_num)

Secret number key generated by B: 5


Once you have the secret number key, now client B will send an encrypted message through symmetric encryption. In order to encrypt it, you can free to use any type of symmetric encryption algorythm. In my case, I will use the characters movement technique with tha ASCII code table.

In [21]:
message = input("Enter the message to be encrypted: ")

encrypted_message = [(ord(char) + secret_key_num) for char in message]
print("Cypher message:", encrypted_message)

Cypher message: [103, 122, 106, 115, 116, 120, 37, 105, 110, 102, 120]


After entering the secret message and encrypting it, you will need to send it to client A.

In [22]:
# Send the encrypted message to Client B
def send_encrypted_message(encrypted_message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(('localhost', 12346))  # Channel 2
        s.sendall(bytes(encrypted_message))
        
        print("Message sent to Client B")

send_encrypted_message(encrypted_message)

Message sent to Client B


After sending the message to client A, he will need to decrypt.